In [ ]:
import numpy as np
import tensorflow as tf
import os 
import cv2
!pip install tflearn
import tflearn 
from tflearn.layers.conv import conv_2d, max_pool_2d 
from tflearn.layers.core import input_data, dropout, fully_connected 
from tflearn.layers.estimator import regression 

#Members:...
#https://blog.paperspace.com/image-captioning-with-ai/
#image preprocessing

# TEST_W_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/test/with_mask"
# TEST_WO_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/test/without_mask"
# TRAIN_W_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/train/with_mask"
# TRAIN_WO_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/train/without_mask"

!git clone https://github.com/oviyajay/ML-Final-Project

#label the images
#create training data and test data

In [ ]:
#Eric 
train_path = "/content/ML-Final-Project/maskdata/maskdata/train/"
os.listdir(train_path)

test_path = "/content/ML-Final-Project/maskdata/maskdata/test/"
os.listdir(test_path)

In [ ]:
label_names = ['with_mask', 'without_mask']
IMG_SIZE = 150
LR = 1e-3
MODEL_NAME = 'withMaskvsnoMask-{}-{}.model'.format(LR, '6conv-basic')

def load_data(path):
  data = []

  for label in label_names:
    for filename in os.listdir(os.path.join(path, label)):
      img_file = os.path.join(path, label, filename)
      # loading the image from the path and then converting them into 
      # greyscale for easier covnet prob 
      img = cv2.imread(img_file, cv2.IMREAD_COLOR)

      # resizing the image for processing them in the covnet 
      img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

      label_id = label_names.index(label)

      # final step-forming the training data list with numpy array of the images 
      data.append([np.array(img), np.array(label_id)]) 

      return data

In [ ]:

train_data = load_data(train_path)
test_data = load_data(test_path)
# print(train_data)

In [ ]:
# train is a list of [x, y] pairs
train_x = [i[0] for i in train_data]  #i[0] is a numpy array that is the size/shape of a single image
train_x = np.array(train_x).reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # num_image, width, height, channels
train_y = [i[1] for i in train_data]
test_x = [i[0] for i in test_data]
test_x = np.array(test_x).reshape(-1, IMG_SIZE, IMG_SIZE, 1) 
test_y = [i[1] for i in test_data]
print(train_x.shape)

In [ ]:
#Iant
#tf.reset_default_graph() 
cnn = input_data(shape =[None, IMG_SIZE, IMG_SIZE, 1], name ='input') 

#convolution and pooling operations
#as for the arguments, (convolutional network, numbers of channels, filter size(if one number 'x' is passed, then both dimensions are 'x' ), activation function used)
#in our case, we're starting with 16 channels, gradually working up to 128. By convention, the number of channels gradually increase or stay the same as it works its way further into the CNN
#filter size changed from 5x5 to 3x3

cnn = conv_2d(cnn, 16, 3, activation ='relu') 
cnn = max_pool_2d(cnn, 3) 

cnn = conv_2d(cnn, 32, 3, activation ='relu') 
cnn = max_pool_2d(cnn, 3) 
  
cnn = conv_2d(cnn, 64, 3, activation ='relu') 
cnn = max_pool_2d(cnn, 3) 
  
cnn = conv_2d(cnn, 128, 3, activation ='relu') 
cnn = max_pool_2d(cnn, 3) 
  
cnn = fully_connected(cnn, 1024, activation ='relu') 
cnn = dropout(cnn, 0.8) 
  
cnn = fully_connected(cnn, 2, activation ='softmax') 
cnn = regression(cnn, optimizer ='adam', learning_rate = LR, 
      loss ='categorical_crossentropy', name ='targets') 
  
model = tflearn.DNN(cnn, tensorboard_dir ='log')

In [ ]:

#Oviya
model.fit({'input': train_x}, {'targets': train_y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}),snapshot_step = 500, show_metric=True,run_id = MODEL_NAME )
model.save(MODEL_NAME)